# Отток клиентов

---
<a id='К_оглавлению'></a>

## Описание проекта

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

---
### Признаки
* **RowNumber** — индекс строки в данных.
* **CustomerId** — уникальный идентификатор клиента.
* **Surname** — фамилия.
* **CreditScore** — кредитный рейтинг.
* **Geography** — страна проживания.
* **Gender** — пол.
* **Age** — возраст.
* **Tenure** — количество недвижимости у клиента.
* **Balance** — баланс на счёте.
* **NumOfProducts** — количество продуктов банка, используемых клиентом.
* **HasCrCard** — наличие кредитной карты.
* **IsActiveMember** — активность клиента.
* **EstimatedSalary** — предполагаемая зарплата.

### Целевой признак
* **Exited** — факт ухода клиента.
---
### Разделим данное исследование на несколько частей

1. [Подготовка данных](#Подготовка_данных)
 * 1.1 Загрузка необходимых библиотек и изучение таблицы.
 * 1.2 Измение названия столбцов и их перевод в нужный регистр.
 * 1.3 Рзаделение таблицы на выборки для `'sklearn'` и `'catboost'`.
 * 1.4 Масштабирование признаков для `'sklearn'`.
2. [Исследование задачи](#Исследование_задачи)
 * 2.1 Обучение и проверка мделей `'sklearn'` на метрике *F1* и *AUC-ROC*.
 * 2.2 Обучение и проверка мделей `'catboost'` на метрике *F1* и *AUC-ROC*.
3. [Борьба с дисбалансом](#Борьба_с_дисбалансом)
 * 3.1 Увеличение выборки с помощью *upsampling*
 * 3.2 Обучение и проверка мделей `'sklearn'` на метрике *F1* и *AUC-ROC*.
  * Подбор порога для логистической регрессиии.
 * 3.3 Обучение и проверка мделей `'catboost'` на метрике *F1* и *AUC-ROC*. 
4. [Тестирование модели](#Тестирование_модели)
 * 4.1 Проверка моделей на тестовой выборке, выбор лучшей модели.
 ---

<a id='Подготовка_данных'></a>
# 1. Подготовка данных
Загрузим необходимые билиотеки и посмотрим на датасет.

In [1]:
import pandas as pd
import numpy as np

# Модели sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
# Тесты
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_auc_score
# Модель catboost
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('/datasets/Churn.csv')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Датасет нуждается в предобработке, для упрощения обучения моделей. Необходимо привести названия к единому виду, заполнить пропуски, а так же избавиться от столбцов не несущих в себе ценности для обучения моделей.

Изменим названия столбцов, а тк же приведем все значения к нижнему регистру.

In [3]:
df = df.rename({'CreditScore':'credit_score', 'NumOfProducts':'num_of_products', 'HasCrCard':'has_cr_card', 
                                'IsActiveMember':'is_active_member', 'EstimatedSalary':'estimated_salary'}, axis=1)
df.columns = df.columns.str.lower()


Избавимся от ненужных столбцов, которые усложнят обучение модели.

In [4]:
df = df.drop(['rownumber', 'customerid', 'surname'], axis=1)
df

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


Проверим, есть ли в столбце `'tenure'` нулевые значения.

In [5]:
df[df['tenure']==0].count()

credit_score        382
geography           382
gender              382
age                 382
tenure              382
balance             382
num_of_products     382
has_cr_card         382
is_active_member    382
estimated_salary    382
exited              382
dtype: int64

Из этого можно сделать вывод, что пропущенные значения не являются отсутствием недвижимости у клиента. Поэтому заполним их медианным значением.

In [6]:
df['tenure'] = df['tenure'].fillna(value=df['tenure'].median()) 

Закодируем таблицу.

In [7]:
# o - One-Hot Encoding
o_df = pd.get_dummies(df, drop_first=True)

Разделим выборки на тренировочную, валидационную и тестовую для `'sklearn'` и `'catboost'`. **60%** тренировочная, **20%** валидационная и **20%** тестовая выборки.

In [8]:
target = o_df['exited']
features = o_df.drop(['exited'], axis=1)

# Разделим выборку на тренировочную и выборку для тестирований.
features_train, features_valid_and_test, target_train, target_valid_and_test = train_test_split(
    features, target, stratify=target, test_size=0.40, random_state=12345)

# разделим выборку тестирований на валидационную и тестовую.
features_test, features_valid, target_test, target_valid = train_test_split(
    features_valid_and_test, target_valid_and_test, test_size=0.50, random_state=12345)

Для обучения `'catboost'` будем использовать выборку без кодирования.

In [9]:
c_target = df['exited']
c_features = df.drop(['exited'], axis=1)

# Разделим выборку на тренировочную и выборку для тестирований.
c_features_train, c_features_valid_and_test, c_target_train, c_target_valid_and_test = train_test_split(
    c_features, c_target, stratify=target, test_size=0.40, random_state=12345)

# разделим выборку тестирований на валидационную и тестовую.
c_features_test, c_features_valid, c_target_test, c_target_valid = train_test_split(
    c_features_valid_and_test, c_target_valid_and_test, test_size=0.50, random_state=12345)

Для корректного обучения моделей `'sklearn'` необходимо масштабировать признаки.

In [10]:
numeric = ['credit_score', 'age', 'tenure', 'balance', 'estimated_salary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

## Вывод

Была проведена предобработка данных, для дольнейшего обучения моделей. Названия столбцов были приведены к единому виду и регистру. Удалены столбцы, которые будут мешать обучению. Данные были разделены на выборки, а так же закодированы.

[К оглавлению](#К_оглавлению)

---

<a id='Исследование_задачи'></a>
# 2. Исследование задачи

Проверим наслоко выборка сбалансирована.

In [11]:
print('Клиентов осталось: {}, Клиентов ушло: {}'.format(len(df['exited']) - sum(df['exited']), sum(df['exited'])))

Клиентов осталось: 7963, Клиентов ушло: 2037


Можно сделать вывод, что есть большой дисбаланс классов.

Обучим модели безучета дисбаланса.

Первой моделью будет **DecisionTreeClassifier**. Автоматизируем процесс подбора нужной глубины.

In [12]:
best_model_one = None
best_result_one = 0
best_debth = 0

for depth in range(1, 25):
    model_one = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_one.fit(features_train, target_train)
    
    predictions_one = model_one.predict(features_valid)
    result_one = f1_score(target_valid, predictions_one)
    
    if result_one > best_result_one:
        best_model_one = model_one
        best_result_one = result_one
        best_debth = depth
print(best_debth)

7


Лучшая глубина - 7. Подставим это значение и посмотрим на значения *F1* и *AUC-ROC*.

In [13]:
model_one = DecisionTreeClassifier(random_state=12345, max_depth=7)
model_one.fit(features_train, target_train)
    
predictions_one = model_one.predict(features_valid)
first_f = f1_score(target_valid, predictions_one)
first_f

0.5846153846153846

In [14]:
probabilities_valid_one = model_one.predict_proba(features_valid)[:,1]
first_roc_auc = roc_auc_score(target_valid, probabilities_valid_one)
first_roc_auc

0.8462955535868211

Обе метрки показывают, что можель достаточно хорошо прогнозирует данные. *AUC-ROC* гораздо выше 0.5. Из этого можно сделать выводы, что модель выучилась и не гадает.

Теперь обучим модель **RandomForestClassifier**, а так же подберемк количество деревьев.

In [15]:
best_result_two = 0
best_est = 0

for est in range(1,25,1):
    model_two = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_two.fit(features_train, target_train)
    
    predicted_valid_two = model_two.predict(features_valid)
    result_two = f1_score(target_valid, predicted_valid_two)
    
    if result_two > best_result_two:
        best_result_two = result_two
        best_depth = est 
        
print(best_result_two)
print(best_depth)

0.5865522174535049
23


Теперь подставим значение количества деревьев и подберем наилучшую глубину.

In [16]:
model_two = RandomForestClassifier(random_state=12345, max_depth=80, n_estimators=23)
model_two.fit(features_train, target_train)
    
predicted_valid_two = model_two.predict(features_valid)

second_f = f1_score(target_valid, predicted_valid_two)
second_f

0.5865522174535049

In [17]:
probabilities_valid_two = model_two.predict_proba(features_valid)[:,1]
second_roc_auc = roc_auc_score(target_valid, probabilities_valid_two)
second_roc_auc

0.8501146322191361

Метрики так же показывают хорошие значения. Лучше, чем в предылущей выброрке.

Обучим моедль **LogisticRegression**.

In [18]:
model_three = LogisticRegression(solver='liblinear',random_state=12345)
model_three.fit(features_train, target_train)

predicted_valid_three = model_three.predict(features_valid)

third_f = f1_score(target_valid, predicted_valid_three)
third_f

0.29616087751371123

Значение *F1* достаточно низкое. Посмотрим на матрицу ошибок.

In [19]:
confusion_matrix(target_valid, predicted_valid_three)

array([[1534,   42],
       [ 343,   81]])

На матрице видно, что модель достаточно сильно ошибается.

In [20]:
probabilities_valid_three = model_three.predict_proba(features_valid)[:,1]
third_roc_auc = roc_auc_score(target_valid, probabilities_valid_three)
third_roc_auc

0.7684084977492578

*AUC-ROC* так же ниже, чем на предыдущих моделях.

Для обучения `'catboost'` обозначим для модели категориальные переменные и добавим их в *'cat_features'*.

In [21]:
cat_features = ['geography', 'gender']

In [22]:
cat_model = CatBoostClassifier(verbose=100,early_stopping_rounds=200, random_state=12345)

In [23]:
cat_model.fit(c_features_train,c_target_train,
          eval_set=(c_features_valid,c_target_valid),
          cat_features=cat_features)

Learning rate set to 0.09418
0:	learn: 0.6239164	test: 0.6243430	best: 0.6243430 (0)	total: 62.7ms	remaining: 1m 2s
100:	learn: 0.2910728	test: 0.3243922	best: 0.3236872 (94)	total: 6.04s	remaining: 53.8s
200:	learn: 0.2513304	test: 0.3299717	best: 0.3236872 (94)	total: 12.1s	remaining: 48s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.323687202
bestIteration = 94

Shrink model to first 95 iterations.


In [24]:
cat_predicted = cat_model.predict(c_features_valid)
forth_f = f1_score(c_target_valid,cat_predicted)
forth_f

0.587719298245614

In [25]:
probabilities_valid_cat = cat_model.predict_proba(c_features_valid)[:,1]
forth_roc_auc = roc_auc_score(c_target_valid, probabilities_valid_cat)
forth_roc_auc

0.8843381859975099

*F1* и *AUC-ROC* показали наилучшие результаты среди обученых моделей.

## Вывод

Модели обучены, гиперпараметры подобраны. Лучше всего себя показал **CatBoostClassifier**, хуже всего **LogisticRegression**.

[К оглавлению](#К_оглавлению)

---
<a id='Борьба_с_дисбалансом'></a>
# 3. Борьба с дисбалансом

Для борьбы с дисбалансом я буду использовать **upsampling**. Автоматизируем этопроцесс.

In [26]:
def unsample(features,target,repeat):
    
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    upsampled_features = pd.concat([features_zeros] + [features_ones] * repeat)
    upsampled_target = pd.concat([target_zeros] + [target_ones] * repeat)
    
    upsampled_features, upsampled_target = shuffle(upsampled_features, upsampled_target, random_state=12345)
    
    return upsampled_features, upsampled_target

Используем функцию **upsample** на обучающих выборках для `'sklearn'` и `'catboost'`.

In [27]:
upsampled_features, upsampled_target = unsample(features_train,target_train,3)

In [28]:
c_upsampled_features, c_upsampled_target = unsample(c_features_train,c_target_train,3)

Заново обучим модели и подберем их параметры, а так же сравним с предыдущими значениями. Начнем с **DecisionTreeClassifier**.

In [29]:
best_model_one = None
best_result_one = 0
best_debth = 0

for depth in range(1, 25):
    model_one = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_one.fit(upsampled_features, upsampled_target)
    
    predictions_one = model_one.predict(features_valid)
    result_one = f1_score(target_valid, predictions_one)
    
    if result_one > best_result_one:
        best_model_one = model_one
        best_result_one = result_one
        best_debth = depth
print(best_debth)

7


In [30]:
model_one = DecisionTreeClassifier(random_state=12345, max_depth=7)
model_one.fit(upsampled_features, upsampled_target)
    
predictions_one = model_one.predict(features_valid)
print('"F1" Первой модели:',first_f)
print('"F1" Модели после апсемплинга:',f1_score(target_valid, predictions_one))

"F1" Первой модели: 0.5846153846153846
"F1" Модели после апсемплинга: 0.6061899679829242


In [31]:
confusion_matrix(target_valid, predictions_one)

array([[1347,  229],
       [ 140,  284]])

In [32]:
probabilities_valid_one = model_one.predict_proba(features_valid)[:,1]
print('"AUC-ROC" Первой модели:',first_roc_auc)
print('"AUC-ROC" Модели после апсемплинга:',roc_auc_score(target_valid, probabilities_valid_one))

"AUC-ROC" Первой модели: 0.8462955535868211
"AUC-ROC" Модели после апсемплинга: 0.8412096841777608


*F1* у модели после даунсемплинга вырос, но *"AUC-ROC"* немного упал. Возможно потому, что данных для обучения стало больше, чем надо. Теперь обучим **RandomForestClassifier**.

In [33]:
best_result_two = 0
best_est = 0

for est in range(1,25,1):
    model_two = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_two.fit(upsampled_features, upsampled_target)
    
    predicted_valid_two = model_two.predict(features_valid)
    result_two = f1_score(target_valid, predicted_valid_two)
    
    if result_two > best_result_two:
        best_result_two = result_two
        best_depth = est 
        
print(best_result_two)
print(best_depth)

0.6319796954314721
13


In [34]:
model_two = RandomForestClassifier(random_state=12345, max_depth=40, n_estimators=13)
model_two.fit(upsampled_features, upsampled_target)
    
predicted_valid_two = model_two.predict(features_valid)
print('"F1" Второй модели:',second_f)
print('"F1" Модели после апсемплинга:',f1_score(target_valid, predicted_valid_two))

"F1" Второй модели: 0.5865522174535049
"F1" Модели после апсемплинга: 0.6319796954314721


In [35]:
confusion_matrix(target_valid, predicted_valid_two)

array([[1461,  115],
       [ 175,  249]])

In [36]:
probabilities_valid_two = model_two.predict_proba(features_valid)[:,1]
print('"AUC-ROC" Второй модели:',second_roc_auc)
print('"AUC-ROC" Модели после апсемплинга:',roc_auc_score(target_valid, probabilities_valid_two))

"AUC-ROC" Второй модели: 0.8501146322191361
"AUC-ROC" Модели после апсемплинга: 0.8456333504932478


Как и в предыдущих моделях *F1* вырос, но качество классификатора упало. Посмотрим на **LogisticRegression**.

In [37]:
model_three = LogisticRegression(solver='liblinear',random_state=12345)
model_three.fit(upsampled_features, upsampled_target)

predicted_valid_three = model_three.predict(features_valid)
print('"F1" Третей модели:',third_f)
print('"F1" Модели после апсемплинга:',f1_score(target_valid, predicted_valid_three))

"F1" Третей модели: 0.29616087751371123
"F1" Модели после апсемплинга: 0.5130784708249497


In [38]:
probabilities_valid_three = model_three.predict_proba(features_valid)[:,1]
print('"AUC-ROC" Третей модели:',third_roc_auc)
print('"AUC-ROC" Модели после апсемплинга:',roc_auc_score(target_valid, probabilities_valid_three))

"AUC-ROC" Третей модели: 0.7684084977492578
"AUC-ROC" Модели после апсемплинга: 0.7724879681065032


У этой модели все значения возросли. Особенно слильно выросла *'F1-мера'*. Модели стало проще ориентироваться в данных. Попробуем изменить проговое значение классификации.

In [39]:
probabilities_valid = model_three.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.4, 0.6, 0.01):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f}, F1 = {:.4f}".format(
        threshold, precision, recall, f1))

Порог = 0.40 | Точность = 0.374, Полнота = 0.745, F1 = 0.4976
Порог = 0.41 | Точность = 0.379, Полнота = 0.731, F1 = 0.4992
Порог = 0.42 | Точность = 0.388, Полнота = 0.724, F1 = 0.5053
Порог = 0.43 | Точность = 0.392, Полнота = 0.705, F1 = 0.5042
Порог = 0.44 | Точность = 0.403, Полнота = 0.696, F1 = 0.5104
Порог = 0.45 | Точность = 0.407, Полнота = 0.684, F1 = 0.5101
Порог = 0.46 | Точность = 0.413, Полнота = 0.667, F1 = 0.5099
Порог = 0.47 | Точность = 0.419, Полнота = 0.656, F1 = 0.5110
Порог = 0.48 | Точность = 0.427, Полнота = 0.637, F1 = 0.5109
Порог = 0.49 | Точность = 0.437, Полнота = 0.625, F1 = 0.5146
Порог = 0.50 | Точность = 0.447, Полнота = 0.601, F1 = 0.5131
Порог = 0.51 | Точность = 0.449, Полнота = 0.566, F1 = 0.5010
Порог = 0.52 | Точность = 0.465, Полнота = 0.557, F1 = 0.5064
Порог = 0.53 | Точность = 0.478, Полнота = 0.540, F1 = 0.5072
Порог = 0.54 | Точность = 0.482, Полнота = 0.517, F1 = 0.4989
Порог = 0.55 | Точность = 0.486, Полнота = 0.500, F1 = 0.4930
Порог = 

Максимального значения 'F1' удалось достичь при пороге в **0.49**.

Проверим результаты у сбалансированной выборки.

In [40]:
model_three = LogisticRegression(solver='liblinear',random_state=12345, class_weight='balanced')
model_three.fit(upsampled_features, upsampled_target)

predicted_valid_three_b = model_three.predict(features_valid)
print('"F1" Третей модели:',third_f)
print('"F1" Модели после апсемплинга:',f1_score(target_valid, predicted_valid_three_b))
print('\n')
probabilities_valid_three_b = model_three.predict_proba(features_valid)[:,1]
print('"AUC-ROC" Третей модели:',third_roc_auc)
print('"AUC-ROC" Модели после апсемплинга:',roc_auc_score(target_valid, probabilities_valid_three))

"F1" Третей модели: 0.29616087751371123
"F1" Модели после апсемплинга: 0.505982905982906


"AUC-ROC" Третей модели: 0.7684084977492578
"AUC-ROC" Модели после апсемплинга: 0.7724879681065032


Результаты модели немного хуже, чем у предыдущей моедели.

Теперь обучим модель **CatBoostClassifier** на новых данных и проверим результат.

In [41]:
cat_model.fit(c_upsampled_features, c_upsampled_target,
          eval_set=(c_features_valid,c_target_valid),
          cat_features=cat_features)

Learning rate set to 0.09762
0:	learn: 0.6476847	test: 0.6429207	best: 0.6429207 (0)	total: 18ms	remaining: 17.9s
100:	learn: 0.3662589	test: 0.3765163	best: 0.3765163 (100)	total: 6.59s	remaining: 58.7s
200:	learn: 0.2952389	test: 0.3671067	best: 0.3671067 (200)	total: 13.6s	remaining: 54.2s
300:	learn: 0.2447200	test: 0.3646046	best: 0.3637681 (272)	total: 20.7s	remaining: 48s
400:	learn: 0.2054667	test: 0.3657286	best: 0.3637681 (272)	total: 27.4s	remaining: 41s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.3637681463
bestIteration = 272

Shrink model to first 273 iterations.


In [42]:
c_cat_predicted = cat_model.predict(c_features_valid)
print('"F1" Четвертой модели:',forth_f)
print('"F1" Модели после апсемплинга:',f1_score(c_target_valid, c_cat_predicted))

"F1" Четвертой модели: 0.587719298245614
"F1" Модели после апсемплинга: 0.6361556064073226


In [43]:
probabilities_valid_cat = cat_model.predict_proba(c_features_valid)[:,1]
print('"AUC-ROC" Четвертой модели:',forth_roc_auc)
print('"AUC-ROC" Модели после апсемплинга:',roc_auc_score(c_target_valid, probabilities_valid_cat))

"AUC-ROC" Четвертой модели: 0.8843381859975099
"AUC-ROC" Модели после апсемплинга: 0.8751915525332821


In [44]:
confusion_matrix(c_target_valid, c_cat_predicted)

array([[1404,  172],
       [ 146,  278]])

Как и в первых двух моделях виден рост *'F1'* и не большое падение *'AUC-ROC'*

## Вывод 

После апсемплинга наблюдается увеличение *'F1-меры'*. Так же у большоинства моделей упало качество классификатора кроме **LogisticRegression**. У логистической регрессии наблюдаектся рост по обоим показателям. Лидер по качеству **CatBoostClassifier**.

[К оглавлению](#К_оглавлению)

---
<a id='Тестирование_модели'></a>
# 4. Тестирование модели

Протестируем все модели на тестовой выборке и найдем лидера среди них.

In [45]:
predictions_one = model_one.predict(features_test)
predicted_valid_two = model_two.predict(features_test)
predicted_valid_three = model_three.predict(features_test)
probabilities_valid_three_b = model_three.predict(features_test)
c_cat_predicted = cat_model.predict(c_features_test)

In [46]:
final_df = pd.DataFrame(data={'DecisionTreeClassifier':[f1_score(target_test, predictions_one)],
                              'RandomForestClassifier':[f1_score(target_test, predicted_valid_three)], 
                              'LogisticRegression':[f1_score(target_test, predicted_valid_three)],
                              'LogisticRegressionBal':[f1_score(target_test, predicted_valid_three_b)],
                              'CatBoostClassifier':[f1_score(c_target_test, c_cat_predicted)]}, index=['F1'])
final_df

,DecisionTreeClassifier,RandomForestClassifier,LogisticRegression,LogisticRegressionBal,CatBoostClassifier
F1,0.555678,0.496872,0.496872,0.26737,0.595948


Среди всех моделей наилучший результат имеет **CatBoostClassifier**.

[К оглавлению](#К_оглавлению)

---
## Общий вывод

Данные были предобработаны, разбиты на выборки, были обучены модели. При дисбалансной выборке лучший результат показал **CatBoostClassifier**. После апсемплинга *'F1-метрика'* у всех моделей возрасла и в большинстве упал показатель *'AUC-ROC'*. Так же сильно возрасли показатели у **LogisticRegression**, но лидером все равно остался `'catboost'`. На финальном тестировании модели **CatBoostClassifier** так же проказал самые высокие значения *'F1'*. 